In [1]:
# Import the required libraries and functions

import numpy as np
import pandas as pd
from keras import initializers, regularizers, constraints
from keras.layers import Dense, Input, LSTM, Bidirectional, GRU, Dropout, Embedding
from keras.models import Sequential, Model, load_model
from keras.initializers import Constant
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
import os

Using TensorFlow backend.


In [0]:
# File paths

TRAIN_FILE_PATH = '../data/news_train.json'
TEST_FILE_PATH = '../data/news_test.json' 
SUBMISSION_FILE_PATH = '../data/samplesubmissionNLP.csv'
GLOVE_FILE_PATH = # Enter glove.6B.100d file path here 

In [0]:
# Loading and preprocessing training data

df = pd.read_json(TRAIN_FILE_PATH, lines=True)
df['concat'] =  df['headline'] + " " + df['short_description'] + " " + df['authors']
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df.concat)
df['concat_tokenized'] = tokenizer.texts_to_sequences(df.concat)
maxlen = 50
X = list(sequence.pad_sequences(df.concat_tokenized, maxlen=maxlen))
df.head()

,authors,category,date,headline,link,short_description,concat,concat_tokenized
0,Melissa Jeltsen,CRIME,2018-05-26,There Were 2 Mass Shootings In Texas Last Week...,https://www.huffingtonpost.com/entry/texas-ama...,She left her husband. He killed their children...,There Were 2 Mass Shootings In Texas Last Week...,"[79, 110, 271, 1398, 3273, 6, 705, 147, 119, 2..."
1,Andy McDonald,ENTERTAINMENT,2018-05-26,Will Smith Joins Diplo And Nicky Jam For The 2...,https://www.huffingtonpost.com/entry/will-smit...,Of course it has a song.,Will Smith Joins Diplo And Nicky Jam For The 2...,"[41, 1066, 3146, 27121, 3, 25416, 5701, 8, 1, ..."
2,Ron Dicker,ENTERTAINMENT,2018-05-26,Hugh Grant Marries For The First Time At Age 57,https://www.huffingtonpost.com/entry/hugh-gran...,The actor and his longtime girlfriend Anna Ebe...,Hugh Grant Marries For The First Time At Age 5...,"[5834, 3907, 9460, 8, 1, 76, 55, 18, 494, 9203..."
3,Ron Dicker,ENTERTAINMENT,2018-05-26,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,https://www.huffingtonpost.com/entry/jim-carre...,The actor gives Dems an ass-kicking for not fi...,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,"[1406, 13821, 3812, 69034, 1670, 12569, 3, 542..."
4,Ron Dicker,ENTERTAINMENT,2018-05-26,Julianna Margulies Uses Donald Trump Poop Bags...,https://www.huffingtonpost.com/entry/julianna-...,"The ""Dietland"" actress said using the bags is ...",Julianna Margulies Uses Donald Trump Poop Bags...,"[31454, 38420, 2004, 83, 37, 8502, 3639, 2, 11..."


In [0]:
# Encoding the categories / assigning a number to each category

categories = df.groupby('category').size().index.tolist()
category_int = {}
int_category = {}
for i, k in enumerate(categories):
    category_int.update({k:i})
    int_category.update({i:k})

df['category_to_id'] = df['category'].apply(lambda x: category_int[x])

In [0]:
# Preparing the embedding index using pretrained GloVe embedding. Source: https://blog.keras.io/using-pre-trained-word-embeddings-in-a-keras-model.html

word_index = tokenizer.word_index
EMBEDDING_DIM = 100
embeddings_index = {}
f = open(GLOVE_FILE_PATH)
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

In [3]:
# Creating the embedding layer. Source: https://blog.keras.io/using-pre-trained-word-embeddings-in-a-keras-model.html

# embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
# for word, i in word_index.items():
#     embedding_vector = embeddings_index.get(word)
#     if embedding_vector is not None:
#         embedding_matrix[i] = embedding_vector

embedding_layer = Embedding(len(word_index)+1,
                            EMBEDDING_DIM,

                            input_length=maxlen,
                            trainable=False,
                            name="Embedding layer")

NameError: ignored

In [0]:
# prepared data 

X = np.array(X)
Y = np_utils.to_categorical(list(df.category_to_id))

In [0]:
# Creating the model

inp = Input(shape=(maxlen,), dtype='int32')
x = embedding_layer(inp)
x = Bidirectional(LSTM(128, return_sequences=True, name="BiLSTM-1"))(x)
x = Dropout(0.5, name="Dropout-1")(x)
x = Bidirectional(LSTM(128, name="BiLSTM-2"))(x)
x = Dropout(0.5, name="Dropout-2")(x)
outp = Dense(len(int_category), activation='softmax', name="FC-layer")(x)
model = Model(inp, outp)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 50)                0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 50, 100)           12238800  
_________________________________________________________________
bidirectional_3 (Bidirection (None, 50, 256)           234496    
_________________________________________________________________
dropout_3 (Dropout)          (None, 50, 256)           0         
_________________________________________________________________
bidirectional_4 (Bidirection (None, 256)               394240    
_________________________________________________________________
dropout_4 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 38)                9766      
Total para

In [0]:
# Training the model

model_history = model.fit(X, Y, batch_size=256, epochs=8, validation_split = 0.2)

In [0]:
# Loading submission file

df2 = pd.read_csv(SUBMISSION_FILE_PATH)
df2 = df2.drop([0,1,2])

# Loading test data

df3 = pd.read_json(TEST_FILE_PATH, lines=True)
df3.head()

,Unnamed: 0,labels
0,0,BUSINESS
1,1,SPORTS
2,2,WORLDPOST


In [0]:
# Preprocessing the test data

df3['text'] =  df3['headline'] + " " + df3['short_description'] + " " + df3['authors']
X = tokenizer.texts_to_sequences(df3.text)
df3['words'] = X
print(df3.head())
maxlen = 50
X = list(sequence.pad_sequences(df3.words, maxlen=maxlen))
X = np.array(X)

,authors,date,headline,link,short_description,text,words
0,"Emily Bennington, Contributor\nAuthor, 'Who Sa...",2012-12-26,Making Sense of a Senseless World,https://www.huffingtonpost.com/entry/sandy-hoo...,I know I speak for many when I say that every ...,Making Sense of a Senseless World I know I spe...,"[257, 711, 4, 5, 12503, 75, 13, 87, 13, 1140, ..."
1,"Andrea Wachter, Contributor\nPsychotherapist a...",2012-12-26,Dog Talk,https://www.huffingtonpost.com/entry/self-este...,Helping my clients learn to accept who they ar...,Dog Talk Helping my clients learn to accept wh...,"[634, 370, 1105, 32, 4060, 556, 2, 2100, 47, 5..."
2,"CafeMom, Contributor\nCafeMom is the meting pl...",2012-12-26,7 Soothing Ways to Beat Stress During Pregnancy,https://www.huffingtonpost.com/entry/stress-du...,If you're pregnant and already feeling stresse...,7 Soothing Ways to Beat Stress During Pregnanc...,"[256, 15858, 192, 2, 1927, 492, 203, 1764, 57,..."
3,"Robin Gelfenbien, Contributor\nWriter, comedia...",2012-12-26,The Hard Truth About Sharing Your Birthday Wit...,https://www.huffingtonpost.com/entry/holiday-b...,"As a kid, my birthday parties weren't always w...",The Hard Truth About Sharing Your Birthday Wit...,"[1, 379, 663, 26, 1964, 20, 1016, 12, 2822, 3,..."
4,,2012-12-26,Optical Illusion And Photobomb Finally Come To...,https://www.huffingtonpost.com/entry/optical-i...,We assume the man in the background looks so m...,Optical Illusion And Photobomb Finally Come To...,"[15842, 7877, 3, 22878, 547, 244, 433, 9, 3748..."


In [0]:
# Making predictions on test data

predictions = model.predict(X, verbose=1)

30854/30854 [==============================] - 97s 3ms/step


In [0]:
# Picking out the category with highest probability

predictions = np.argmax(predictions, axis=-1)
predictions.shape

(30854,)

In [0]:
# Writing predicitions to submissions file

for i in range(predictions.shape[0]):
  df2 = df2.append({'Unnamed: 0' : i, 'labels' : int_category[predictions[i]]} , ignore_index=True)
  if(i%100==0):
    print(i)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
18

In [0]:
# Saving submissions file to GDrive

df2.to_csv('nlp2.csv')